<a href="https://colab.research.google.com/github/mohammad2682/FVS-SVM-RKHS/blob/main/SVM_RKHS_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.metrics import confusion_matrix
from sklearn import datasets

In [4]:
N_m=5

In [5]:
import sklearn
A, y=sklearn.datasets.make_classification(n_samples=20,n_features=2,n_informative=2,n_redundant=0,weights=[0.8],random_state=5)
display(A)
display(y)


array([[-2.88470459, -3.09334521],
       [-0.38012753,  0.50960898],
       [ 0.08828312,  0.2825261 ],
       [-1.08727082,  0.91207538],
       [ 0.2095121 ,  1.87770417],
       [-1.26292912,  0.70597421],
       [-1.02649626,  0.67572811],
       [-0.9910247 , -0.92034095],
       [ 0.7250405 ,  1.11196313],
       [-0.82696232,  1.19851322],
       [-1.89255856,  0.40341791],
       [-2.87618154, -3.52767119],
       [-1.20026133, -1.15842117],
       [ 0.04623351,  0.31605039],
       [ 0.2248898 ,  2.1926224 ],
       [ 0.62833321, -2.45446179],
       [-1.87794153, -2.33296202],
       [-1.08536788,  0.91049297],
       [ 0.68698129, -1.99674589],
       [-1.00349992, -1.13101638]])

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0])

In [6]:
def sigma(A):
    N = A.shape[0]
    sigma = []
    for i in range(N):
        for j in range(i+1, N):
            sigma.append(np.square(np.linalg.norm(A[i,:]-A[j,:])))

    return 0.1 * np.max(sigma)

In [7]:
sig=sigma(A)
print(sig)

4.233840225882515


In [8]:
def K(FV1, FV2):
    dist = np.linalg.norm(FV1-FV2)
    return np.exp(-1*np.square(dist)/(2*sig))

In [9]:
def give_KSS(S):
    KSS = []
    for i in S:
        for j in S:
            KSS.append( K(A[i,:], A[j,:]))

    return np.array(KSS).reshape((len(S), len(S)))

In [10]:
give_KSS([1,5])

array([[1.        , 0.90792768],
       [0.90792768, 1.        ]])

In [11]:
def give_KSx(S, x):
    KSx = []
    for i in S:
        kx = K(A[i,:], x)
        KSx.append(kx)
    return np.array(KSx).reshape((len(S), 1))

In [12]:
def give_beta(S, x, sig):
  KSx = give_KSx(S, x)
  KSS = give_KSS(S)
  DSS = np.diag(np.diag(np.sqrt(KSS)))
  beta = np.dot(np.dot(np.transpose(KSx), np.linalg.inv(KSS)), DSS)
  return beta

In [33]:
def give_landa(df, minor_cls, major_cls):
  minor = df.iloc[minor_cls, :]
  major = df.iloc[major_cls, :]
  landa = []
  for col in df.columns[:-1]:
    mean_minor = minor[col].mean()
    mean_major = major[col].mean()
    var_minor = minor[col].var()
    var_major = major[col].var()
    landa_i = np.square(mean_minor - mean_major)/(var_minor - var_major)
    landa.append(landa_i)
  return landa

In [13]:
display(give_KSx([1,5],x=A[6,:]))

array([[0.94876077],
       [0.99331279]])

In [14]:
minor_index=np.where(y==1)[0].tolist()
lambdamax=-np.inf
best_iter=-1
total_S=[]
for i in minor_index:
    current_S=[i]
    ctrl=1
    while (len(current_S)<N_m and ctrl==1):
        current_inv =np.linalg.inv(give_KSS(current_S))
        max_LF=-np.inf
        max_LF_index=-1
        remaining=list(set(range(A.shape[0]))-set(current_S))
        #print(remaining)
        for j in remaining:
            KSx=give_KSx(current_S,A[j,:])
            LF =abs(1-((KSx.T).dot(current_inv).dot(KSx)))
            #print(LF)
            if LF> max_LF:
                max_LF=LF
                max_LF_index=j
        if(max_LF<0.001):
            ctrl=0
        else:
            current_S.append(max_LF_index)
    total_S.append(current_S)  
    

In [44]:
major_index = list(set(range(A.shape[0]))-set(minor_index))
landa_max = -1000
for S in total_S:
  flag = 0
  for vector in A:
    beta = give_beta(S, vector, sig = sig)
    if flag == 0:
      B = beta
      flag = -1
      continue
    B = np.concatenate((B, beta))

  B_df = pd.DataFrame(B)
  B_df["Class"] = y

  landa = give_landa(B_df, minor_index, major_index)
  landa_t = np.sum(landa)/len(S)

  if landa_t > landa_max:
    #print(landa_t)
    landa_max = landa_t
    best_S = S
    best_B = B_df.copy()

0.026019926609763287
0.04560025275662379
